# Federated Learning Training Plan: Host Plan & Model

Here we load Plan and Model params created earlier in "Create Plan" notebook, host them to PyGrid, 
and run sample syft.js app that executes them.  

In [2]:
import asyncio
import websockets
import json
import binascii
import base64

async def sendWsMessage(data):
    async with websockets.connect(gatewayWsUrl) as websocket:
        await websocket.send(json.dumps(data))
        message = await websocket.recv()
        return json.loads(message)

## Step 5a: Host in PyGrid

**NOTE**: Right now PyGrid supports only 1 version of plan (list of ops or torchscript).
But Plans will converge to one type here https://github.com/OpenMined/PySyft/issues/2994#issuecomment-595333791.

Here we load "ops list" Plan.

In [3]:
# Load files with protobuf created in "Create Plan" notebook.
with open("model_params.pb", "rb") as model_file, open("tp_ts.pb", "rb") as plan_file: 
    serialized_model = model_file.read()
    serialized_ops_plan = plan_file.read()


Follow PyGrid README.md to build `openmined/grid-gateway` image from the latest `dev` branch 
and spin up PyGrid using `docker-compose up`.

In [4]:
# Default gateway address when running locally 
gatewayWsUrl = "ws://127.0.0.1:5000"

This hand-crafted request emulates `pygrid.host_federated_training` from the roadmap.

In [5]:
# These name/version you use in worker
name = "mnist"
version = "1.0.2"

host_request = {
    "type": "federated/host-training",
    "data": {
        "model": binascii.hexlify(serialized_model).decode("utf-8"),
        "plans": {
            "training_plan": binascii.hexlify(serialized_ops_plan).decode("utf-8"),
            # "extra_plan": ...,
        },
        "protocols": {
            # "secure_agg_protocol": ...,
        },
        "averaging_plan": "",
        "client_config": {
            "name": name,  
            "version": version,
            "batch_size": 64,
            "lr": 0.01,
            "max_updates": 100  # custom syft.js option that limits number of training loops per worker
        },
        "server_config": {
            "min_workers": 3,  # temporarily this plays role "min # of worker's diffs" for triggering cycle end event
            "max_workers": 3,
            "pool_selection": "random",
            "num_cycles": 5,
            "do_not_reuse_workers_until_cycle": 4,
            "cycle_length": 28800,
            "minimum_upload_speed": 0,
            "minimum_download_speed": 0
        }
    }
}

Shoot!

If everything's good, success is returned.
If the name/version already exists in PyGrid, change them above or cleanup PyGrid db by re-creating docker containers. 


In [7]:
response = await sendWsMessage(host_request)
print("Host response:", response)

Host response: {'type': 'federated/host-training', 'data': {'status': 'success'}}


Let's double-check that data is loaded by requesting a cycle.

In [8]:
auth_request = {
    "type": "federated/authenticate",
    "data": {}
}
auth_response = await sendWsMessage(auth_request)
print('Auth response: ', json.dumps(auth_response, indent=2))

cycle_request = {
    "type": "federated/cycle-request",
    "data": {
        "worker_id": auth_response['data']['worker_id'],
        "model": name,
        "version": version,
        "ping": 1,
        "download": 1000,
        "upload": 1000,
    }
}
cycle_response = await sendWsMessage(cycle_request)
print('Cycle response:', json.dumps(cycle_response, indent=2))


Auth response:  {
  "type": "federated/authenticate",
  "data": {
    "status": "success",
    "worker_id": "365b8750-20e2-4351-91fe-de2f2e93f33c"
  }
}
Cycle response: {
  "type": "federated/cycle-request",
  "data": {
    "status": "accepted",
    "request_key": "35f9fbc6f2d3b85aede92bd15f0dfdcaa40b0cc5ccb13b138071737ce227bd5d",
    "model": "mnist",
    "plans": {
      "training_plan": 2
    },
    "protocols": {},
    "client_config": {
      "name": "mnist",
      "version": "1.0.2",
      "batch_size": 64,
      "lr": 0.01,
      "max_updates": 100
    },
    "model_id": 1
  }
}


## Step 6a: Train

Start and open "with-grid" example in syft.js project (http://localhost:8080 by default), 
enter model name and version and start FL training.



## Step 7a: Submit diff

This emulates submitting worker's diff (created earlier in Execute Plan notebook) to PyGrid.
After several diffs submitted, PyGrid will end the cycle and create new model checkpoint and cycle. 

In [7]:
with open("diff.pb", "rb") as f:
    diff = f.read()

report_request = {
    "type": "federated/report",
    "data": {
        "worker_id": auth_response['data']['worker_id'],
        "request_key": cycle_response['data']['request_key'],
        "diff": base64.b64encode(diff).decode("utf-8")
    }
}

report_response = await sendWsMessage(report_request)
print('Report response:', json.dumps(report_response, indent=2)) 

Report response: {
  "type": "federated/report",
  "data": {
    "status": "success"
  }
}
